In [48]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
import mlflow
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [33]:
# Chargement des données
chemin_dossier = "C:/Users/paulm/Documents/Projet 7/Projet7withCSV/data/" 
df = pd.read_csv(os.path.join(chemin_dossier, 'processed_data.csv'))

Preparation :

In [34]:
# Suppression des colonnes avec trop de valeurs uniques
colonnes_a_supprimer = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']
df.drop(columns=colonnes_a_supprimer, inplace=True)

# Sélectionner uniquement les colonnes numériques
colonnes_numeriques = df.select_dtypes(include=[np.number])

# Identifier les colonnes contenant des valeurs infinies
colonnes_infinies = colonnes_numeriques.columns.to_series()[np.isinf(colonnes_numeriques).any()]
# Identifier les lignes contenant des valeurs infinies
lignes_avec_infinis = df.index[np.isinf(colonnes_numeriques).any(axis=1)]
# Suppression des lignes contenant des valeurs infinies
df.drop(lignes_avec_infinis, inplace=True)

# Encodage des variables catégorielles avec OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(ordinal_encoder.fit_transform(df), columns=df.columns)

# Créer un imputeur avec la stratégie de votre choix (mean, median, most_frequent)
imputer = SimpleImputer(strategy='mean') 

# Appliquer l'imputation sur les données numériques
colonnes_numeriques = df_encoded.select_dtypes(include=[np.number])
df_encoded[colonnes_numeriques.columns] = imputer.fit_transform(colonnes_numeriques)

In [36]:
print(df_encoded['SK_ID_CURR'])
print(df['SK_ID_CURR'])

0              0.0
1              1.0
2              2.0
3              3.0
4              4.0
            ...   
307487    307487.0
307488    307488.0
307489    307489.0
307490    307490.0
307491    307491.0
Name: SK_ID_CURR, Length: 307492, dtype: float64
0         100002
1         100003
2         100004
3         100006
4         100007
           ...  
307506    456251
307507    456252
307508    456253
307509    456254
307510    456255
Name: SK_ID_CURR, Length: 307492, dtype: int64


Création du modèle :

In [40]:
X = df_encoded.drop(columns=['TARGET'])  
y = df_encoded['TARGET']

# Définir l'URL de suivi de MLflow
mlflow_url = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(mlflow_url)

# Configurer l'expérience MLflow
mlflow.set_experiment("Credit Scoring Experiment")

# Fractionner les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir votre modèle XGBoost
model = xgb.XGBClassifier()

# Entraîner votre modèle XGBoost
model.fit(X_train, y_train)

# Enregistrer le modèle dans un fichier .pkl à l'aide de joblib
model_path = 'model/xgboost_model.pkl'
os.makedirs(os.path.dirname(model_path), exist_ok=True)
joblib.dump(model, model_path)

# Utiliser MLflow pour suivre votre modèle
with mlflow.start_run(run_name="XGBoost Model"):
    # Loguer les paramètres du modèle
    mlflow.log_params({
        'n_estimators': model.get_params()['n_estimators'],
        'max_depth': model.get_params()['max_depth'],
        'learning_rate': model.get_params()['learning_rate']
    })

    # Faire des prédictions sur l'ensemble de test
    y_pred = model.predict(X_test)

    # Afficher le rapport de classification et la matrice de confusion
    classification_report_str = classification_report(y_test, y_pred)
    confusion_matrix_str = confusion_matrix(y_test, y_pred)

    print("Rapport de classification :")
    print(classification_report_str)

    print("Matrice de confusion :")
    print(confusion_matrix_str)

    # Loguer les métriques et les résultats dans MLflow
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_text(classification_report_str, "classification_report.txt")
    mlflow.log_text(str(confusion_matrix_str), "confusion_matrix.txt")

    # Évaluation du modèle avec le meilleur seuil pour minimiser le coût métier
    y_prob = model.predict_proba(X_test)[:, 1]
    thresholds = np.linspace(0, 1, 100)
    costs = []
    cost_fp = 1
    cost_fn = 10
    for threshold in thresholds:
        y_pred_thresholded = (y_prob > threshold).astype(int)
        fp = np.sum((y_pred_thresholded == 1) & (y_test == 0)) * cost_fp
        fn = np.sum((y_pred_thresholded == 0) & (y_test == 1)) * cost_fn
        total_cost = fp + fn
        costs.append(total_cost)

    best_threshold = thresholds[np.argmin(costs)]
    print(f"Meilleur seuil pour minimiser le coût métier : {best_threshold}")

    y_pred_best_threshold = (y_prob > best_threshold).astype(int)

    # Affichage du rapport de classification et de la matrice de confusion avec le meilleur seuil
    classification_report_str_best = classification_report(y_test, y_pred_best_threshold)
    confusion_matrix_str_best = confusion_matrix(y_test, y_pred_best_threshold)

    print("Rapport de classification avec le meilleur seuil :")
    print(classification_report_str_best)

    print("Matrice de confusion avec le meilleur seuil :")
    print(confusion_matrix_str_best)

    # Loguer les métriques et les résultats avec le meilleur seuil dans MLflow
    mlflow.log_metric("best_threshold", best_threshold)
    mlflow.log_text(classification_report_str_best, "classification_report_best_threshold.txt")
    mlflow.log_text(str(confusion_matrix_str_best), "confusion_matrix_best_threshold.txt")

    # Loguer le modèle dans MLflow
    mlflow.sklearn.log_model(model, "model")

# Afficher le chemin du modèle sauvegardé
print(f"Modèle enregistré à : {model_path}")

Rapport de classification :
              precision    recall  f1-score   support

         0.0       0.92      0.99      0.96     56609
         1.0       0.46      0.07      0.12      4890

    accuracy                           0.92     61499
   macro avg       0.69      0.53      0.54     61499
weighted avg       0.89      0.92      0.89     61499

Matrice de confusion :
[[56220   389]
 [ 4561   329]]
Meilleur seuil pour minimiser le coût métier : 0.08080808080808081
Rapport de classification avec le meilleur seuil :
              precision    recall  f1-score   support

         0.0       0.96      0.74      0.83     56609
         1.0       0.18      0.66      0.28      4890

    accuracy                           0.73     61499
   macro avg       0.57      0.70      0.56     61499
weighted avg       0.90      0.73      0.79     61499

Matrice de confusion avec le meilleur seuil :
[[41655 14954]
 [ 1661  3229]]
Modèle enregistré à : model/xgboost_model.pkl


In [44]:
#Séparation des caractéristiques et de la cible, division en ensembles d'entraînement et de test :
X = df_encoded.drop(columns=['TARGET'])
y = df_encoded['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

mlflow_url = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(mlflow_url)
mlflow.set_experiment("Credit Scoring Experiment")

model = XGBClassifier(scale_pos_weight=10, random_state=42)
params = {'n_estimators': [50, 100], 'max_depth': [3, 6, 9]}

In [43]:
with mlflow.start_run(run_name="XGBoost"):
    grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='f1', cv=3)
    grid_search.fit(X_train, y_train)

    mlflow.log_params(grid_search.best_params_)
    print(f"Meilleurs paramètres pour XGBoost: {grid_search.best_params_}")
    print(f"Score F1 moyen sur le jeu de validation: {grid_search.best_score_:.3f}")

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Évaluation du modèle avec le meilleur seuil pour minimiser le coût métier
    y_prob = best_model.predict_proba(X_test)[:, 1]
    thresholds = np.linspace(0, 1, 100)
    costs = []
    cost_fp = 1
    cost_fn = 10
    for threshold in thresholds:
        y_pred_thresholded = (y_prob > threshold).astype(int)
        fp = np.sum((y_pred_thresholded == 1) & (y_test == 0)) * cost_fp
        fn = np.sum((y_pred_thresholded == 0) & (y_test == 1)) * cost_fn
        total_cost = fp + fn
        costs.append(total_cost)

    best_threshold = thresholds[np.argmin(costs)]
    print(f"Meilleur seuil pour minimiser le coût métier avec XGBoost: {best_threshold}")

    y_pred_best_threshold = (y_prob > best_threshold).astype(int)

    # Affichage du rapport de classification et de la matrice de confusion
    classification_report_str = classification_report(y_test, y_pred_best_threshold)
    confusion_matrix_str = confusion_matrix(y_test, y_pred_best_threshold)

    print(f"Rapport de classification avec le meilleur seuil pour XGBoost:")
    print(classification_report_str)

    print(f"Matrice de confusion avec le meilleur seuil pour XGBoost:")
    print(confusion_matrix_str)

    mlflow.log_metric("best_score_f1", grid_search.best_score_)
    mlflow.log_metric("best_threshold", best_threshold)
    mlflow.log_text(classification_report_str, "classification_report.txt")
    mlflow.log_text(str(confusion_matrix_str), "confusion_matrix.txt")

    # Loguer le modèle
    model_path = 'model/xgboost_model.pkl'
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    joblib.dump(best_model, model_path)
    mlflow.sklearn.log_model(best_model, "model")


Meilleurs paramètres pour XGBoost: {'max_depth': 6, 'n_estimators': 100}
Score F1 moyen sur le jeu de validation: 0.302
Meilleur seuil pour minimiser le coût métier avec XGBoost: 0.4141414141414142
Rapport de classification avec le meilleur seuil pour XGBoost:
              precision    recall  f1-score   support

         0.0       0.96      0.72      0.82     56609
         1.0       0.17      0.67      0.27      4890

    accuracy                           0.72     61499
   macro avg       0.57      0.69      0.55     61499
weighted avg       0.90      0.72      0.78     61499

Matrice de confusion avec le meilleur seuil pour XGBoost:
[[40812 15797]
 [ 1620  3270]]


In [49]:
# Entraînement du modèle avec GridSearchCV et validation croisée
with mlflow.start_run(run_name="XGBoost"):
    print("Entraînement du modèle XGBoost avec validation croisée...")

    # Utilisation de KFold pour la validation croisée
    kf = KFold(n_splits=3, shuffle=True, random_state=42)

    grid_search = GridSearchCV(estimator=model, param_grid=params, scoring='f1', cv=kf)
    grid_search.fit(X_train, y_train)

    mlflow.log_params(grid_search.best_params_)
    print(f"Meilleurs paramètres pour XGBoost: {grid_search.best_params_}")
    print(f"Score F1 moyen sur le jeu de validation: {grid_search.best_score_:.3f}")

    # Évaluation du modèle avec les meilleures paramètres sur le jeu de test
    best_model = grid_search.best_estimator_

    # Prédiction avec validation croisée
    y_pred_cv = cross_val_predict(best_model, X_test, y_test, cv=kf)

    # Calcul des métriques sur les prédictions
    f1_cv = f1_score(y_test, y_pred_cv)
    classification_report_cv = classification_report(y_test, y_pred_cv)
    confusion_matrix_cv = confusion_matrix(y_test, y_pred_cv)

    print(f"Score F1 sur le jeu de test avec XGBoost et validation croisée : {f1_cv:.3f}")
    print(f"Rapport de classification sur le jeu de test avec XGBoost et validation croisée :")
    print(classification_report_cv)
    print(f"Matrice de confusion sur le jeu de test avec XGBoost et validation croisée :")
    print(confusion_matrix_cv)

    mlflow.log_metric("f1_score_cv", f1_cv)
    mlflow.log_text(classification_report_cv, "classification_report_cv.txt")
    mlflow.log_text(str(confusion_matrix_cv), "confusion_matrix_cv.txt")

    # Sauvegarde du modèle
    model_path = 'model/xgboost_model_cv.pkl'
    os.makedirs(os.path.dirname(model_path), exist_ok=True)
    joblib.dump(best_model, model_path)
    mlflow.sklearn.log_model(best_model, "model_cv")

Entraînement du modèle XGBoost avec validation croisée...
Meilleurs paramètres pour XGBoost: {'max_depth': 6, 'n_estimators': 100}
Score F1 moyen sur le jeu de validation: 0.303
Score F1 sur le jeu de test avec XGBoost et validation croisée : 0.261
Rapport de classification sur le jeu de test avec XGBoost et validation croisée :
              precision    recall  f1-score   support

         0.0       0.94      0.90      0.92     56609
         1.0       0.22      0.32      0.26      4890

    accuracy                           0.86     61499
   macro avg       0.58      0.61      0.59     61499
weighted avg       0.88      0.86      0.87     61499

Matrice de confusion sur le jeu de test avec XGBoost et validation croisée :
[[51179  5430]
 [ 3343  1547]]
